# 05b: CNN-LSTM Model Design and Training 
This notebook defines and trains a hybrid CNN-LSTM model for ICU mortality prediction. The workflow includes:

- Loading and validating preprocessed data
- Defining a robust CNN-LSTM architecture
- Using early stopping and model checkpointing
- Saving the trained model and training history

In [ ]:
# Imports and Configuration
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout, Concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Load prepared data 
data = np.load('../data/processed/cnn_lstm_data.npz', allow_pickle=True)
X_train, y_train = data['X_train'], data['y_train']
X_val, y_val = data['X_val'], data['y_val']
X_test, y_test = data['X_test'], data['y_test']
static_train, static_val, static_test = data['static_train'], data['static_val'], data['static_test']

# print shapes to confirm alignment
print('X_train shape:', X_train.shape)
print('static_train shape:', static_train.shape)
print('y_train shape:', y_train.shape)
print('X_val shape:', X_val.shape)
print('static_val shape:', static_val.shape)
print('y_val shape:', y_val.shape)
print('X_test shape:', X_test.shape)
print('static_test shape:', static_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (4406, 202, 24)
static_train shape: (4406, 9)
y_train shape: (4406,)
X_val shape: (640, 202, 24)
static_val shape: (640, 9)
y_val shape: (640,)
X_test shape: (800, 202, 24)
static_test shape: (800, 9)
y_test shape: (800,)


## 1. Model Architecture

CNN layers extract local patterns; LSTM layers capture temporal dependencies. Static features can be concatenated before the output layer.

## 2. Load Prepared Data

Load the processed data arrays from the previous data preparation notebook. Confirm shapes and preview data for sanity check.

In [ ]:
# Load prepared data
data = np.load('../data/processed/cnn_lstm_data.npz', allow_pickle=True)
X_train, y_train = data['X_train'], data['y_train']
X_val, y_val = data['X_val'], data['y_val']
X_test, y_test = data['X_test'], data['y_test']
static_train, static_val, static_test = data['static_train'], data['static_val'], data['static_test']

# Sanity check
print('X_train shape:', X_train.shape)
print('static_train shape:', static_train.shape)
print('y_train shape:', y_train.shape)

# Model definition
seq_input = Input(shape=(X_train.shape[1], X_train.shape[2]), name='seq_input')
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(1e-4))(seq_input)
x = Dropout(0.3)(x)
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.3)(x)
x = LSTM(64, return_sequences=False, kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.3)(x)

static_input = Input(shape=(static_train.shape[1],), name='static_input')
concat = Concatenate()([x, static_input])
dense = Dense(32, activation='relu', kernel_regularizer=l2(1e-4))(concat)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[seq_input, static_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'Precision', 'Recall'])
model.summary()

X_train shape: (4406, 202, 24)
static_train shape: (4406, 9)
y_train shape: (4406,)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ seq_input           │ (None, 202, 24)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 202, 64)   │      4,672 │ seq_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 202, 64)   │          0 │ conv1d_8[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 202, 64)   │     12,352 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 202, 64)   │          0 │ conv1d_9[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 64)        │     33,024 │ dropout_13[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 64)        │          0 │ lstm_4[0][0]      │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ static_input        │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 73)        │          0 │ dropout_14[0][0], │
│ (Concatenate)       │                   │            │ static_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      2,368 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │         33 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 52,449 (204.88 KB)

 Trainable params: 52,449 (204.88 KB)

 Non-trainable params: 0 (0.00 B)

## 2. Model Training

Use early stopping and model checkpointing to prevent overfitting.

## 3. Model Architecture: CNN-LSTM Hybrid

- **Input 1:** Time series (sequence) features (shape: [batch, timesteps, features])
- **Input 2:** Static features (shape: [batch, static_features])
- **CNN layers:** Extract local temporal patterns
- **LSTM layer:** Capture long-term dependencies
- **Dense layers:** Combine sequence and static features
- **Output:** Binary mortality prediction (sigmoid)

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('cnn_lstm_best_model.h5', save_best_only=True)
]

history = model.fit(
    [X_train, static_train], y_train,
    validation_data=([X_val, static_val], y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks
)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ seq_input           │ (None, 202, 24)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 202, 64)   │      4,672 │ seq_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 202, 64)   │          0 │ conv1d_10[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 202, 64)   │     12,352 │ dropout_15[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 202, 64)   │          0 │ conv1d_11[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 64)        │     33,024 │ dropout_16[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 64)        │          0 │ lstm_5[0][0]      │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ static_input        │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 73)        │          0 │ dropout_17[0][0], │
│ (Concatenate)       │                   │            │ static_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 32)        │      2,368 │ concatenate_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         33 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 52,449 (204.88 KB)

 Trainable params: 52,449 (204.88 KB)

 Non-trainable params: 0 (0.00 B)

...

69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - AUC: 0.9972 - Precision: 0.9723 - Recall: 0.9986 - loss: 0.0575 - val_AUC: 0.9978 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0741
Epoch 66/100
Epoch 66/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - AUC: 0.9971 - Precision: 0.9730 - Recall: 0.9986 - loss: 0.0573 - val_AUC: 0.9978 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0757
Epoch 67/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - AUC: 0.9971 - Precision: 0.9730 - Recall: 0.9986 - loss: 0.0573 - val_AUC: 0.9978 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0757
Epoch 67/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - AUC: 0.9971 - Precision: 0.9767 - Recall: 0.9987 - loss: 0.0567

69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9971 - Precision: 0.9767 - Recall: 0.9987 - loss: 0.0568 - val_AUC: 0.9978 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0720
Epoch 68/100
Epoch 68/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - AUC: 0.9972 - Precision: 0.9730 - Recall: 0.9987 - loss: 0.0566 - val_AUC: 0.9978 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0733
Epoch 69/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - AUC: 0.9972 - Precision: 0.9730 - Recall: 0.9987 - loss: 0.0566 - val_AUC: 0.9978 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0733
Epoch 69/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - AUC: 0.9972 - Precision: 0.9734 - Recall: 0.9992 - loss: 0.0559 - val_AUC: 0.9979 - val_Precision: 0.8241 - val_Recall: 1.0000 - val_loss: 0.0749
Epoch 70/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - AUC: 0.9972 - Precision: 0.9734 - Recall: 0.9992 - loss: 0.0559 - val_AUC: 0.9979 - val_Precision: 0.8241 - val_Recall: 1.0000 - val_loss: 0.

69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 88ms/step - AUC: 0.9974 - Precision: 0.9714 - Recall: 0.9989 - loss: 0.0543 - val_AUC: 0.9979 - val_Precision: 0.8396 - val_Recall: 1.0000 - val_loss: 0.0703
Epoch 73/100
Epoch 73/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - AUC: 0.9972 - Precision: 0.9741 - Recall: 0.9978 - loss: 0.0548 - val_AUC: 0.9979 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0723
Epoch 74/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - AUC: 0.9972 - Precision: 0.9741 - Recall: 0.9978 - loss: 0.0548 - val_AUC: 0.9979 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0723
Epoch 74/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - AUC: 0.9971 - Precision: 0.9768 - Recall: 0.9986 - loss: 0.0550 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.0736
Epoch 75/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - AUC: 0.9971 - Precision: 0.9768 - Recall: 0.9986 - loss: 0.0550 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.

69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - AUC: 0.9972 - Precision: 0.9718 - Recall: 0.9982 - loss: 0.0544 - val_AUC: 0.9980 - val_Precision: 0.8396 - val_Recall: 1.0000 - val_loss: 0.0693
Epoch 76/100
Epoch 76/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9972 - Precision: 0.9748 - Recall: 0.9986 - loss: 0.0536 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.0726
Epoch 77/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9972 - Precision: 0.9748 - Recall: 0.9986 - loss: 0.0536 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.0726
Epoch 77/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9972 - Precision: 0.9763 - Recall: 0.9989 - loss: 0.0534 - val_AUC: 0.9980 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0711
Epoch 78/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9972 - Precision: 0.9763 - Recall: 0.9989 - loss: 0.0534 - val_AUC: 0.9980 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.

69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 88ms/step - AUC: 0.9973 - Precision: 0.9748 - Recall: 0.9984 - loss: 0.0526 - val_AUC: 0.9980 - val_Precision: 0.8396 - val_Recall: 1.0000 - val_loss: 0.0690
Epoch 81/100
Epoch 81/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - AUC: 0.9973 - Precision: 0.9748 - Recall: 0.9987 - loss: 0.0522 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.0739
Epoch 82/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - AUC: 0.9973 - Precision: 0.9748 - Recall: 0.9987 - loss: 0.0522 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.0739
Epoch 82/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - AUC: 0.9973 - Precision: 0.9752 - Recall: 0.9989 - loss: 0.0518 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.0712
Epoch 83/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - AUC: 0.9973 - Precision: 0.9752 - Recall: 0.9989 - loss: 0.0518 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.

## 4. Model Training

Use early stopping and model checkpointing to prevent overfitting. Monitor validation loss and AUC.

In [ ]:
# Save model and training history
model.save('cnn_lstm_final_model.h5')
with open('cnn_lstm_training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("Model and training history saved.")

Epoch 1/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - AUC: 0.9973 - Precision: 0.9781 - Recall: 0.9989 - loss: 0.0509

69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - AUC: 0.9973 - Precision: 0.9781 - Recall: 0.9989 - loss: 0.0510 - val_AUC: 0.9980 - val_Precision: 0.8396 - val_Recall: 1.0000 - val_loss: 0.0689
Epoch 2/100
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9973 - Precision: 0.9771 - Recall: 0.9991 - loss: 0.0511 - val_AUC: 0.9980 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0692
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9973 - Precision: 0.9771 - Recall: 0.9991 - loss: 0.0511 - val_AUC: 0.9980 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0692
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - AUC: 0.9973 - Precision: 0.9758 - Recall: 0.9982 - loss: 0.0509

69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - AUC: 0.9973 - Precision: 0.9758 - Recall: 0.9982 - loss: 0.0509 - val_AUC: 0.9980 - val_Precision: 0.8476 - val_Recall: 1.0000 - val_loss: 0.0679
Epoch 4/100
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - AUC: 0.9973 - Precision: 0.9771 - Recall: 0.9984 - loss: 0.0509 - val_AUC: 0.9980 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0694
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - AUC: 0.9973 - Precision: 0.9771 - Recall: 0.9984 - loss: 0.0509 - val_AUC: 0.9980 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0694
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9974 - Precision: 0.9797 - Recall: 0.9992 - loss: 0.0503 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.0704
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - AUC: 0.9974 - Precision: 0.9797 - Recall: 0.9992 - loss: 0.0503 - val_AUC: 0.9980 - val_Precision: 0.8165 - val_Recall: 1.0000 - val_loss: 0.0704


69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - AUC: 0.9974 - Precision: 0.9765 - Recall: 0.9989 - loss: 0.0496 - val_AUC: 0.9981 - val_Precision: 0.8476 - val_Recall: 1.0000 - val_loss: 0.0658
Epoch 10/100
Epoch 10/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 8s 109ms/step - AUC: 0.9974 - Precision: 0.9761 - Recall: 0.9989 - loss: 0.0489 - val_AUC: 0.9981 - val_Precision: 0.8476 - val_Recall: 1.0000 - val_loss: 0.0671
Epoch 11/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 8s 109ms/step - AUC: 0.9974 - Precision: 0.9761 - Recall: 0.9989 - loss: 0.0489 - val_AUC: 0.9981 - val_Precision: 0.8476 - val_Recall: 1.0000 - val_loss: 0.0671
Epoch 11/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 8s 106ms/step - AUC: 0.9974 - Precision: 0.9782 - Recall: 0.9989 - loss: 0.0479 - val_AUC: 0.9981 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss: 0.0682
Epoch 12/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 8s 106ms/step - AUC: 0.9974 - Precision: 0.9782 - Recall: 0.9989 - loss: 0.0479 - val_AUC: 0.9981 - val_Precision: 0.8318 - val_Recall: 1.0000 - val_loss

Model and training history saved.
